In [1]:
import pandas as pd 

In [2]:
try:
    ev_pop_df = pd.read_csv('Electric_Vehicle_Population_Data_20240920.csv')
    zipcounty_df = pd.read_csv('ZIP-COUNTY-FIPS_2017-06.csv')
    sizehist_df = pd.read_csv('Electric_Vehicle_Population_Size_History.csv')
    stationlocs_df = pd.read_csv('alt_fuel_stations (Sep 21 2024).csv')
    powerplants_df = pd.read_csv('Power_Plants.csv')
except:
    print('A dataset is missing from pwd')

In [3]:
# ZIP-COUNTY-FIPS_2017-06.csv Processing 
# get only the ZIPs in WA
wa_zip = zipcounty_df.loc[zipcounty_df['STATE'] == 'WA']
wa_zip_cpy = wa_zip.copy()
wa_zip_cpy.drop(['CLASSFP', 'STATE'], inplace=True, axis=1)

wa_zips_rename_map = {
    'ZIP': 'Postal Code',
    'COUNTYNAME': 'County',
    'STCOUNTYFP': 'Area Code',
}

# Using rename method with the dictionary
wa_zip_cpy = wa_zip_cpy.rename(columns=wa_zips_rename_map) 
# create dictionary of all the zipcodes that belong to specific county
washzip_dict = wa_zip.groupby('ZIP')['COUNTYNAME'].apply(list).to_dict()
wa_zip_cpy.columns
print(wa_zip_cpy.columns, wa_zip_cpy.iloc[0,:])

Index(['Postal Code', 'County', 'Area Code'], dtype='object') Postal Code           99125
County         Adams County
Area Code             53001
Name: 49222, dtype: object


In [4]:
# Electric_Vehicle_Population_Data_20240920.csv Processing  
# underscorify
ev_pop_df.columns = ev_pop_df.columns.str.replace(' ', '_')

# good practice
ev_df = ev_pop_df.copy()

# two word names make into one a
ev_df.Make = ev_df.Make.str.replace(' ', '')

# drop NAs
columns_to_check = ['Legislative_District', 'Model', 'County', 'City', 'Electric_Utility', '2020_Census_Tract', 'Vehicle_Location', 'Electric_Range', 'Base_MSRP']
ev_df = ev_df.dropna(subset=columns_to_check)

# Extract longitude (first coordinate) and latitude (second coordinate) from 'POINT' data
ev_df['Longitude'] = ev_df['Vehicle_Location'].str.extract(r'POINT \(([-\d\.]+) [-\d\.]+\)')
ev_df['Latitude'] = ev_df['Vehicle_Location'].str.extract(r'POINT \([-\d\.]+ ([-\d\.]+)\)')

# Drop Vehicle Location Column
ev_df.drop(['Vehicle_Location', 'Base_MSRP', 'VIN_(1-10)'], inplace=True, axis=1)

# convert Postal code into INT
ev_df['Postal_Code'] = ev_df['Postal_Code'].astype(int)
print(ev_df.columns, ev_df.iloc[0,:])

Index(['County', 'City', 'State', 'Postal_Code', 'Model_Year', 'Make', 'Model',
       'Electric_Vehicle_Type',
       'Clean_Alternative_Fuel_Vehicle_(CAFV)_Eligibility', 'Electric_Range',
       'Legislative_District', 'DOL_Vehicle_ID', 'Electric_Utility',
       '2020_Census_Tract', 'Longitude', 'Latitude'],
      dtype='object') County                                                                                Kitsap
City                                                                                 Seabeck
State                                                                                     WA
Postal_Code                                                                            98380
Model_Year                                                                              2023
Make                                                                                  TOYOTA
Model                                                                             RAV4 PRIME
Electric_Vehic

In [5]:
# Electric_Vehicle_Population_Size_History.csv Processing

# Check Duplicates
print(sizehist_df.loc[sizehist_df.duplicated()])
print("No Duplicates")
print()

# check for null values
sizehist_df.isnull().sum()
print('No Null Values')
print()
print(sizehist_df.columns, sizehist_df.iloc[0,:])

Empty DataFrame
Columns: [Date, Plug-In Hybrid Electric Vehicle (PHEV) Count, Battery Electric Vehicle (BEV) Count, Electric Vehicle (EV) Total]
Index: []
No Duplicates

No Null Values

Index(['Date', 'Plug-In Hybrid Electric Vehicle (PHEV) Count',
       'Battery Electric Vehicle (BEV) Count', 'Electric Vehicle (EV) Total'],
      dtype='object') Date                                            January 31 2017
Plug-In Hybrid Electric Vehicle (PHEV) Count               7688
Battery Electric Vehicle (BEV) Count                      14741
Electric Vehicle (EV) Total                               22429
Name: 0, dtype: object


In [6]:
# alt_fuel_stations (Sep 21 2024).csv Processing
print(stationlocs_df.shape)

# No duplicates
stationlocs_df.loc[stationlocs_df.duplicated()]

useless_cols = ['Plus4', 'BD Blends', 'NG Fill Type Code', 'NG PSI', 'EV Other Info', 'Hydrogen Status Link', 
                'NG Vehicle Class', 'LPG Primary', 'E85 Blender Pump', 'Intersection Directions (French)', 
                'Access Days Time (French)', 'BD Blends (French)', 'Hydrogen Is Retail', 'CNG Dispenser Num', 
                'CNG On-Site Renewable Source', 'CNG Total Compression Capacity', 'CNG Storage Capacity', 
                'LNG On-Site Renewable Source', 'E85 Other Ethanol Blends', 'EV Pricing (French)', 'LPG Nozzle Types', 
                'Hydrogen Pressures', 'Hydrogen Standards', 'CNG Fill Type Code', 'CNG PSI', 'CNG Vehicle Class', 
                'LNG Vehicle Class', 'RD Blends', 'RD Blends (French)', 'RD Blended with Biodiesel', 'RD Maximum Biodiesel Level', 
                'CNG Station Sells Renewable Natural Gas', 'LNG Station Sells Renewable Natural Gas', 'Funding Sources', 'Street Address',
               'Intersection Directions', 'Station Phone', 'Expected Date', 'Cards Accepted', 'EV Level1 EVSE Num', 'EV Level2 EVSE Num',
               'EV DC Fast Count', 'EV Network Web', 'Owner Type Code', 'Federal Agency ID', 'Federal Agency Name', 'Access Detail Code',
               'Federal Agency Code','Facility Type', 'EV Pricing', 'EV On-Site Renewable Source', 'Restricted Access', 'NPS Unit Name', 
               'Maximum Vehicle Class']

# drop columns with no values for all rows
stationlocs_df = stationlocs_df.drop(useless_cols, axis=1)

# scorify variable names
stationlocs_df.columns = stationlocs_df.columns.str.replace(' ', '_')
# show columns with missing values to determine which to drop
print(stationlocs_df.columns, stationlocs_df.iloc[0,:])

(2705, 75)
Index(['Fuel_Type_Code', 'Station_Name', 'City', 'State', 'ZIP', 'Status_Code',
       'Groups_With_Access_Code', 'Access_Days_Time', 'EV_Network',
       'Geocode_Status', 'Latitude', 'Longitude', 'Date_Last_Confirmed', 'ID',
       'Updated_At', 'Open_Date', 'EV_Connector_Types', 'Country',
       'Groups_With_Access_Code_(French)', 'Access_Code',
       'EV_Workplace_Charging'],
      dtype='object') Fuel_Type_Code                                                   ELEC
Station_Name                        City of Lacey - City Hall Parking
City                                                            Lacey
State                                                              WA
ZIP                                                             98503
Status_Code                                                         E
Groups_With_Access_Code                                        Public
Access_Days_Time                                       24 hours daily
EV_Network            

In [7]:
# Power_Plants.csv Processing
wa_powerplants_df = powerplants_df.loc[powerplants_df['State'] == 'Washington' ,:]
wa_powerplants_df.drop(['Street_Address'], inplace=True, axis=1)
print(wa_powerplants_df.columns, wa_powerplants_df.iloc[0,:])

Index(['X', 'Y', 'OBJECTID', 'Plant_Code', 'Plant_Name', 'Utility_ID',
       'Utility_Name', 'sector_name', 'City', 'County', 'State', 'Zip',
       'PrimSource', 'source_desc', 'tech_desc', 'Install_MW', 'Total_MW',
       'Bat_MW', 'Bio_MW', 'Coal_MW', 'Geo_MW', 'Hydro_MW', 'HydroPS_MW',
       'NG_MW', 'Nuclear_MW', 'Crude_MW', 'Solar_MW', 'Wind_MW', 'Other_MW',
       'Source', 'Period', 'Longitude', 'Latitude'],
      dtype='object') X                                     -13621609.4909
Y                                       5955096.4539
OBJECTID                                       23631
Plant_Code                                        99
Plant_Name                              Frederickson
Utility_ID                                     15500
Utility_Name                  Puget Sound Energy Inc
sector_name                         Electric Utility
City                                          Tacoma
County                                        Pierce
State                     

C:\Users\naran\AppData\Local\Temp\ipykernel_20100\2577350356.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  wa_powerplants_df.drop(['Street_Address'], inplace=True, axis=1)


In [8]:
# # have to drop columns where ZIP Code is 'OOS' (out of state) so I can convert ZIP Code to int for mapping to work w/o err 
# ev_df.drop(ev_df[ev_df['ZIP_Code'] == 'OOS'].index, inplace=True) 

# # converting zip to int 
# ev_df.ZIP_Code = ev_df.ZIP_Code.astype(int)

# # create new column 'County' in fuel df using county dictionary
# ev_df['County'] = ev_df['ZIP_Code'].map(calizip_dict)

# # since a vehicle can be registered in multiple county, we will just take the first to simplify
# ev_df['County'] = ev_df['County'][0][0]